## Finance Data EDA

In [1]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta
import numpy as np
import plotly.express as px


In [40]:
def get_stock_price(ticker: str):
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.info
    price = info.get("currentPrice") or info.get("regularMarketPrice")
    currency = info.get("currency", "")
    return price, currency

def historical_prices(ticker: str):
    df = yf.download(ticker, period="2y", interval="1d")
    df.reset_index(inplace=True)
    # convert from multi-index to single index
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    # round float values to 2 digits
    df = df.round(2)
    

    return df

In [41]:
ticker = "AAPL"
curent_price, currency = get_stock_price(ticker)
price_history = historical_prices(ticker)

/var/folders/vw/g3z5zv0x57j2tvdkn1nrdrxr0000gn/T/ipykernel_1921/2352154978.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [42]:
# Date descending order 
price_history = price_history.sort_values(by='Date', ascending=False)
price_history

,Date,Close AAPL,High AAPL,Low AAPL,Open AAPL,Volume AAPL
500,2025-07-22,214.40,214.65,212.23,213.24,44917713
499,2025-07-21,212.48,215.78,211.63,212.10,51377400
498,2025-07-18,211.18,211.79,209.70,210.87,48974600
497,2025-07-17,210.02,211.80,209.59,210.57,48068100
496,2025-07-16,210.16,212.40,208.64,210.30,47490500
...,...,...,...,...,...,...
4,2023-07-28,193.89,194.68,192.22,192.74,48291400
3,2023-07-27,191.30,195.25,190.64,194.08,47460200
2,2023-07-26,192.57,193.70,191.40,191.75,47471900
1,2023-07-25,191.70,192.51,191.01,191.41,37283200


In [64]:

fig = px.line(price_history, x="Date", y=f"Close {ticker}", title=f"{ticker} Price History")
fig.show()

In [67]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x=price_history['Date'],
                                     open=price_history[f'Open {ticker}'],
                                     high=price_history[f'High {ticker}'],
                                     low=price_history[f'Low {ticker}'],
                                     close=price_history[f'Close {ticker}'])])
fig.update_layout(title=f"{ticker} Stock Price History (Candlestick Chart)",
                  xaxis_title="Date",
                  yaxis_title=f"Price ({currency})",
                  xaxis_rangeslider_visible=True)
fig.show()
#st.plotly_chart(fig, use_container_width=True)

In [48]:
price_history.columns

Index(['Date', 'Close AAPL', 'High AAPL', 'Low AAPL', 'Open AAPL',
       'Volume AAPL'],
      dtype='object')

In [53]:
# --- 2. Calculate Moving Averages (20‑day & 50‑day) ---
price_history["SMA_20"] = price_history[f"Close {ticker}"].rolling(window=20).mean()
price_history["SMA_50"] = price_history[f"Close {ticker}"].rolling(window=50).mean()

# --- 3. Calculate RSI (14‑period) ---
delta = price_history[f"Close {ticker}"].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
price_history["RSI_14"] = 100 - (100 / (1 + rs))

# --- 4. Calculate MACD (12,26,9) ---
ema12 = price_history[f"Close {ticker}"].ewm(span=12, adjust=False).mean()
ema26 = price_history[f"Close {ticker}"].ewm(span=26, adjust=False).mean()
price_history["MACD"] = ema12 - ema26
price_history["MACD_Signal"] = price_history["MACD"].ewm(span=9, adjust=False).mean()
price_history["MACD_Hist"] = price_history["MACD"] - price_history["MACD_Signal"]
price_history = price_history.round(2)
# --- 5. Fetch the latest real‑time price ---
latest_price = yf.Ticker(ticker).fast_info["lastPrice"]

# # Display last 5 rows with indicators + latest price for context
# display_columns = ["Date", "Adj Close", "SMA_20", "SMA_50", "RSI_14", "MACD", "MACD_Signal", "MACD_Hist"]
# tools.display_dataframe_to_user("TSLA_Indicators_Last5", df[display_columns].tail())

# print(f"Latest TSLA price (real‑time): ${latest_price:.2f}")


In [54]:
price_history

,Date,Close AAPL,High AAPL,Low AAPL,Open AAPL,Volume AAPL,SMA_20,SMA_50,RSI_14,MACD,MACD_Signal,MACD_Hist
500,2025-07-22,214.40,214.65,212.23,213.24,44917713,NaN,NaN,NaN,0.00,0.00,0.00
499,2025-07-21,212.48,215.78,211.63,212.10,51377400,NaN,NaN,NaN,-0.15,-0.03,-0.12
498,2025-07-18,211.18,211.79,209.70,210.87,48974600,NaN,NaN,NaN,-0.38,-0.10,-0.28
497,2025-07-17,210.02,211.80,209.59,210.57,48068100,NaN,NaN,NaN,-0.64,-0.21,-0.43
496,2025-07-16,210.16,212.40,208.64,210.30,47490500,NaN,NaN,NaN,-0.82,-0.33,-0.49
...,...,...,...,...,...,...,...,...,...,...,...,...
4,2023-07-28,193.89,194.68,192.22,192.74,48291400,180.25,177.80,88.32,3.97,1.96,2.01
3,2023-07-27,191.30,195.25,190.64,194.08,47460200,181.08,178.11,79.06,4.10,2.39,1.71
2,2023-07-26,192.57,193.70,191.40,191.75,47471900,181.73,178.49,79.36,4.25,2.76,1.49
1,2023-07-25,191.70,192.51,191.01,191.41,37283200,182.52,178.88,75.31,4.25,3.06,1.19


In [ ]:
# plot1: Price with SMAs
import plotly.graph_objects as go

# Show more days by sorting ascending and plotting the full range
price_history_sorted = price_history.sort_values(by='Date', ascending=True)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=price_history_sorted['Date'],
    y=price_history_sorted[f'Close {ticker}'],
    mode='lines',
    name=f'Close {ticker}'
))
fig.add_trace(go.Scatter(
    x=price_history_sorted['Date'],
    y=price_history_sorted['SMA_20'],
    mode='lines',
    name='20‑day SMA'
))
fig.add_trace(go.Scatter(
    x=price_history_sorted['Date'],
    y=price_history_sorted['SMA_50'],
    mode='lines',
    name='50‑day SMA'
))
fig.update_layout(
    title=f"{ticker} Price with 20‑day & 50‑day SMAs (Full History)",
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    legend_title="Legend",
    xaxis=dict(rangeslider=dict(visible=True))  # Add range slider for zooming
)


fig.show()  

In [69]:
# plot 2: RSI (Relative Strength Index)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=price_history_sorted['Date'],
    y=price_history_sorted['RSI_14'],
    mode='lines',
    name='RSI 14'
))
fig.update_layout(
    title=f"{ticker} RSI 14 (Full History)",
    xaxis_title="Date",
    yaxis_title="RSI",
    legend_title="Legend",
    xaxis=dict(rangeslider=dict(visible=True))  # Add range slider for zoomming
)

fig.add_hline(y=30, line_dash="dash", line_color="red", annotation_text="RSI 30", annotation_position="top left")
fig.add_hline(y=70, line_dash="dash", line_color="green", annotation_text="RSI 70", annotation_position="top left")

#shape the area axline=30 and annotation_text it as "Oversold" which show the label "Oversold Area/Buy Zone"    
fig.add_shape(type="rect",
              x0=price_history_sorted['Date'].min(),
              x1=price_history_sorted['Date'].max(),
              y0=0,
              y1=30,
              fillcolor="red",
              opacity=0.2,
              line_width=0,
              name="Oversold Area/Buy Zone",
              layer="below")    

fig.add_shape(type="rect",
              x0=price_history_sorted['Date'].min(),
              x1=price_history_sorted['Date'].max(),
              y0=70,
              y1=100,
              fillcolor="green",
              opacity=0.2,
              line_width=0,
              name="Overbought Area/Sell Zone",
              layer="below")    
# add annotation_text for shaded area
fig.add_annotation(
    x=price_history_sorted['Date'].max(),
    y=15,
    text="Oversold Area/Buy Zone",
    showarrow=False,
    font=dict(color="red")
)
fig.add_annotation(
    x=price_history_sorted['Date'].max(),
    y=85,
    text="Overbought Area/Sell Zone",
    showarrow=False,
    font=dict(color="green")
)   

# hover text showing the close price and RSI value
fig.update_traces(hovertemplate="<b>Date:</b> %{x}<br><b>Close Price:</b> %{y}<br><b>RSI:</b> %{customdata[0]}",
                  customdata=price_history_sorted[['Close ' + ticker, 'RSI_14']].values)
fig.show()

In [71]:
# plot the MACD (Moving Average Convergence Divergence)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=price_history_sorted['Date'],
    y=price_history_sorted['MACD'],
    mode='lines',
    name='MACD'
))
fig.add_trace(go.Scatter(
    x=price_history_sorted['Date'],
    y=price_history_sorted['MACD_Signal'],
    mode='lines',
    name='MACD Signal'
))
fig.add_trace(go.Bar(
    x=price_history_sorted['Date'],
    y=price_history_sorted['MACD_Hist'],
    name='MACD Histogram',
    marker_color='rgba(0, 0, 255, 0.5)',
    opacity=0.5
))      
fig.update_layout(
    title=f"{ticker} MACD (Full History)",
    xaxis_title="Date",
    yaxis_title="MACD",
    legend_title="Legend",
    xaxis=dict(rangeslider=dict(visible=True))  # Add range slider for zoomming
)
# hover text showing the MACD, MACD Signal, MACD Histogram values and close price 
fig.update_traces(hovertemplate="<b>Date:</b> %{x}<br><b>Close Price:</b> %{customdata[0]}<br><b>MACD:</b> %{y}<br><b>MACD Signal:</b> %{customdata[1]}<br><b>MACD Histogram:</b> %{customdata[2]}",
                  customdata=price_history_sorted[['Close ' + ticker, 'MACD', 'MACD_Signal', 'MACD_Hist']].values)
# move the legend to the top left corner
fig.update_layout(legend=dict(x=0, y=1, traceorder='normal',
                            orientation='h', bgcolor='rgba(0,0,0,0)', bordercolor='rgba(0,0,0,0)'))
fig.show()  

In [ ]:
# plot 3: MACD
